In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pymysql
from selenium.webdriver.common.by import By 

In [2]:
driver  = webdriver.Chrome()

In [3]:
driver.get('https://www.aladin.co.kr/home/welcome.aspx')

In [10]:
driver.find_element(By.XPATH, '//*[contains(text(), "이달의 주목도서")]').click()

In [11]:
soup = bs(driver.page_source, 'html.parser')

In [20]:
book_list = soup.find_all(
    'div', 
    attrs= {
        'class' : 'event_c_item'
    }
)

In [26]:
book_list[0]

<div class="event_c_item"><dl><dt><a href="https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=323558091"><img alt="" src="https://image.aladin.co.kr/product/32355/80/cover500/k022935461_1.jpg" width="100%"/></a></dt>
<dd><div class="event_c_item_sp1">
<!-- 상품 간략히 보기 --> <!--display:inline;-->
<table border="0" cellpadding="0" cellspacing="0" class="" style=";" summary="상품정보: 백년다리" width="100%">
<tbody><tr class="iteminfo_v2tr2">
<td class="iteminfo_v2td1t1td2" style="text-align:left" valign="top">
<a class="bov" href="/shop/wproduct.aspx?ItemId=323558091" target="_self" title="'백년다리' 상품정보보기">백년다리</a>
<br/><span class="gw">다쓰미 이치로 지음, 김향아 옮김<br/></span>
</td>
</tr>
</tbody></table>
</div>
<a href="https://www.aladin.co.kr/shop/wproduct.aspx?ItemId=323558091"><span class="tit-ph">수술 없이 무릎 통증 해방</span>무릎 수술을 잘 하기로 유명하지만 절대로 바로 수술을 해주지 않는 의사, '상식에서 벗어난' 것으로 화제가 된 일본의 정형외과 의사가 쓴 책이다. 이 책은 정형외과 교과서에는 없는 그만의 보존 치료법과 그 실천법을 중점적으로 다루고 있다. 이뿐 아니라, 우리가 무병장수하기 위해 실천하고 개선해야 할 식습관, 사고방식까지 소개하고 있다.</

In [25]:
# 알라딘 이달의주목도서 책 이미지
book_list[0].img['src']

'https://image.aladin.co.kr/product/32355/80/cover500/k022935461_1.jpg'

In [31]:
# 알라딘 이달의 주목도서 책 이름
book_list[0].find(
    'a', 
    attrs = {
        'class' : 'bov'
    }
).get_text()

'백년다리'

In [30]:
# 알라딘 이달의 주목도서 책 저자
book_list[0].find(
    'span', 
    attrs = {
        'class' : 'gw'
    }
).get_text()

'다쓰미 이치로 지음, 김향아 옮김'

In [32]:
book_info = []
for i in book_list:
    book_img = i.img['src']
    book_name = i.find('a', attrs={'class' : 'bov'}).get_text()
    book_writer = i.find('span', attrs={'class':'gw'}).get_text()
    data = [book_img, book_name, book_writer]
    book_info.append(data)

In [33]:
book_info

[['https://image.aladin.co.kr/product/32355/80/cover500/k022935461_1.jpg',
  '백년다리',
  '다쓰미 이치로 지음, 김향아 옮김'],
 ['https://image.aladin.co.kr/product/32420/23/cover500/k112935993_1.jpg',
  '푸틴을 죽이는 완벽한 방법',
  '김진명 지음'],
 ['https://image.aladin.co.kr/product/32355/15/cover500/k342935468_2.jpg',
  '소녀 동지여 적을 쏴라',
  '아이사카 토마 지음, 이소담 옮김'],
 ['https://image.aladin.co.kr/product/32314/50/cover500/k922834152_1.jpg',
  '어린이를 위한 데일 카네기의 인간관계론',
  '김지연 지음, 유영근 그림'],
 ['https://image.aladin.co.kr/product/32450/93/cover500/k912935391_1.jpg',
  '챗GPT 영어 질문법',
  '일간 소울영어(레바 김) 지음'],
 ['https://image.aladin.co.kr/product/32359/69/cover500/k972935568_2.jpg',
  '네 안에 잠든 거인을 깨워라 (30주년 기념...',
  '토니 로빈스 지음, 홍석윤 옮김'],
 ['https://image.aladin.co.kr/product/32309/15/cover500/8933872264_1.jpg',
  '결심이 필요한 순간들',
  '러셀 로버츠 지음, 이지연 옮김'],
 ['https://image.aladin.co.kr/product/32043/46/cover500/k682834883_2.jpg',
  '향수가 된 식물들',
  '장 클로드 엘레나 지음, 카린 도어링 프로저 그림, 이주영 옮김'],
 ['https://image.aladin.co.kr/product/32197/22

In [34]:
_db = pymysql.connect(
    host = 'localhost', 
    user = 'root', 
    password = 'java', 
    db = 'os', 
    port = 3306
)
cursor = _db.cursor(pymysql.cursors.DictCursor)


In [41]:
# DB에 알라딘 테이블이 존재하면 패스 존재하지 않으면 테이블 생성

sql = """
    CREATE 
    TABLE 
    IF NOT 
    EXISTS 
    aladin(
        name varchar(64) primary key, 
        img TEXT, 
        writer varchar(64)
    )
    
"""

cursor.execute(sql)
_db.commit()

In [44]:
sql  = """
    INSERT 
    INTO 
    aladin 
    (img, name, writer)
    VALUES 
    (%s, %s, %s)
"""
for i in book_info:
    print(i)
    cursor.execute(sql, i)

_db.commit()

['https://image.aladin.co.kr/product/32355/80/cover500/k022935461_1.jpg', '백년다리', '다쓰미 이치로 지음, 김향아 옮김']
['https://image.aladin.co.kr/product/32420/23/cover500/k112935993_1.jpg', '푸틴을 죽이는 완벽한 방법', '김진명 지음']
['https://image.aladin.co.kr/product/32355/15/cover500/k342935468_2.jpg', '소녀 동지여 적을 쏴라', '아이사카 토마 지음, 이소담 옮김']
['https://image.aladin.co.kr/product/32314/50/cover500/k922834152_1.jpg', '어린이를 위한 데일 카네기의 인간관계론', '김지연 지음, 유영근 그림']
['https://image.aladin.co.kr/product/32450/93/cover500/k912935391_1.jpg', '챗GPT 영어 질문법', '일간 소울영어(레바 김) 지음']
['https://image.aladin.co.kr/product/32359/69/cover500/k972935568_2.jpg', '네 안에 잠든 거인을 깨워라 (30주년 기념...', '토니 로빈스 지음, 홍석윤 옮김']
['https://image.aladin.co.kr/product/32309/15/cover500/8933872264_1.jpg', '결심이 필요한 순간들', '러셀 로버츠 지음, 이지연 옮김']
['https://image.aladin.co.kr/product/32043/46/cover500/k682834883_2.jpg', '향수가 된 식물들', '장 클로드 엘레나 지음, 카린 도어링 프로저 그림, 이주영 옮김']
['https://image.aladin.co.kr/product/32197/22/cover500/k472834424_3.jpg', '상황과 이야기', '비비언 고닉 지